In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch import nn, optim
import os
import config
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import random
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import cohen_kappa_score
# from efficientnet_pytorch import EfficientNet
from dataset import DRDataset
import datetime
from torchvision.utils import save_image
from earlystopper import EarlyStopper
import csv 
from utilscopy import (
    load_checkpoint,
    save_checkpoint,
    check_accuracy,
    make_prediction,
    
)

In [3]:
class PerformanceMetric:
    def __init__(self,actual_labels,predicted_labels):
        ''' Initialization of variables '''
        self.actual_labels = actual_labels
        self.predicted_labels = predicted_labels
    def single_value_conversion(self):
        ''' This function is used for Converting model predicted values into single values
           Ex: model_predicted_value: [0,1,0,0,0] and it converts as 1'''
        predicted_labels = self.predicted_labels > 0.5
        prediction_ordinal = np.empty(predicted_labels.shape, dtype = int)
        prediction_ordinal[:,4] = predicted_labels[:,4]
        for i in range(3, -1, -1): prediction_ordinal[:, i] = np.logical_or(predicted_labels[:,i], prediction_ordinal[:,i+1])
        self.predicted_labels = prediction_ordinal.sum(axis = 1)-1
        self.actual_labels = self.actual_labels.sum(axis = 1)-1
    def confusionMatrix(self):
        ''' This function is used for calculating confusion matrix between model predicted values and true values using sklearn implementation.'''
        confusion_matrix_ = confusion_matrix(self.actual_labels, self.predicted_labels)
        return confusion_matrix_
    def precision(self, matrix):
        ''' This function is used for calculating precision matrix between predicted values and true values using confusion matrix'''
        precision_matrix =(((matrix.T)/(matrix.sum(axis=1))).T)
        return precision_matrix
    def recall(self, matrix):
        ''' this function is used for calculating recall matrix between predicted values and true values using confusion matrix'''
        recall_matrix =(matrix/matrix.sum(axis=0))
        return recall_matrix
    def subplot_(self, matrix, i, title):
        ''' This function is used for subplots'''
        plt.subplot(1,3,i)
        labels = [1,2,3,4,5]
        sns.heatmap(matrix, annot=True, cmap=sns.light_palette('green'),linewidths = 0.8,cbar = False, fmt=".3f", xticklabels=labels, yticklabels=labels)
        plt.title(title)
        plt.xlabel('Predicted Class Labels')
        plt.ylabel('Actual Class Labels')
    def plotting(self):
        """
        This function is used for calculating number of misclassified points, confusion, recall and precision matrixes and plotting it using subplots.
        """
        self.single_value_conversion()
        confusion_matrix = self.confusionMatrix()
        #print("Number of misclassified points: ",(len(self.actual_labels)-np.trace(confusion_matrix))/len(self.actual_labels)*100,"\n")
        precision_matrix = self.precision(confusion_matrix)
        recall_matrix = self.recall(confusion_matrix)
        plt.figure(figsize=(20,5))
        self.subplot_(confusion_matrix, 1, 'Confusion Matrix')
        self.subplot_(precision_matrix, 2, 'Precision')
        self.subplot_(recall_matrix, 3, 'Recall')
        plt.show()

In [4]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


seed = 42
set_seed(seed)

Random seed set as 42


In [5]:
def train_one_epoch(loader, model, optimizer, loss_fn, scaler, device):
    losses = []
    loop = tqdm(loader)
    for batch_idx, (data, targets, _) in enumerate(loop):
        
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        with torch.cuda.amp.autocast():
            scores = model(data)
            loss = loss_fn(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

    print(f"Loss average over epoch: {sum(losses)/len(losses)}")

In [6]:
hyperparameter_combinations = [
{'batch_size': 64, 'learning_rate': 0.0002, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 2e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.005, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0005, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 5e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.003, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0003, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 3e-05, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.001, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 0.0001, 'epochs': 700},
{'batch_size': 64, 'learning_rate': 1e-05, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.002, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.0002, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 2e-05, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.005, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.0005, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 5e-05, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.003, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.0003, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 3e-05, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.001, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 0.0001, 'epochs': 700},
{'batch_size': 32, 'learning_rate': 1e-05, 'epochs': 700},
]
#131,64,3e-05,0,82.90636042402826,78.10476751030018,0

In [7]:

from torchvision import models

def training_main(hyperparams,early_stopper):
    BATCH_SIZE = hyperparams['batch_size']
    LEARNING_RATE = hyperparams['learning_rate']
    NUM_EPOCHS = hyperparams['epochs']
    
    best_accuracy = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    best_train_accuracy = 0.0
    best_train_precision = 0.0
    best_train_recall = 0.0
    best_train_f1 = 0.0
        
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_folder = f"Model_weights/DenseNet201_RealSynth_{current_time}_{BATCH_SIZE}_{LEARNING_RATE}"
    os.makedirs(output_folder)
    

    train_ds = DRDataset(
        images_folder="/home/shashank/All_Code/Research/Dataset/APTOS_RS_192/Real_Synthetic/RS_train_split",
        path_to_csv="/home/shashank/All_Code/Research/Dataset/APTOS_RS_192/Real_Synthetic/RS_train_split.csv",
        transform=config.val_transforms,
    )
    val_ds = DRDataset(
        images_folder="/home/shashank/All_Code/Research/Dataset/APTOS_RS_192/Real_Synthetic/RS_val_split",
        path_to_csv="/home/shashank/All_Code/Research/Dataset/APTOS_RS_192/Real_Synthetic/RS_val_split.csv",
        transform=config.val_transforms,
    )
    # test_ds = DRDataset(
    #     images_folder="/home/shashank/All_code/test/images_resized_192",
    #     path_to_csv="/home/shashank/All_code/Research/APTOS_dataset/aptos2019_blindness_detection/test.csv",
    #     transform=config.val_transforms,
    #     train=False,
    # )
    # test_loader = DataLoader(
    #     test_ds, batch_size=BATCH_SIZE, num_workers=6, shuffle=False
    # )
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=True,
    )
   
    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        num_workers=config.NUM_WORKERS, 
        pin_memory=config.PIN_MEMORY,
        shuffle=True,
    )
    
    loss_fn = nn.CrossEntropyLoss()
    
    ########### MODEL ################
    
    model = models.densenet201(pretrained=True)
    num_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        
        nn.Flatten(),
        nn.Dropout(0.5),
        nn.Linear(num_features, 5)
    )
    
    
    #model = models.resnet50(pretrained=True)
    #model.fc = nn.Linear(model.fc.in_features, 5)
    
    # model = models.alexnet(pretrained=True)
    # model.classifier[6] = nn.Linear(model.classifier[6].in_features, 5)
    
    model = model.to(config.DEVICE)
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler()

    if config.LOAD_MODEL and config.CHECKPOINT_FILE in os.listdir():
        load_checkpoint(torch.load(config.CHECKPOINT_FILE), model, optimizer,LEARNING_RATE)
        print("Model Loaded !!!!!!!!")

    # Run after training is done and you've achieved good result
    # on validation set, then run train_blend.py file to use information
    # about both eyes concatenated
    #get_csv_for_blend(val_loader, model, "../train/val_blend.csv")
    #get_csv_for_blend(train_loader, model, "../train/train_blend.csv")
    #get_csv_for_blend(test_loader, model, "../train/test_blend.csv")
    #make_prediction(model, test_loader, "submission.csv")
    #import sys
    #sys.exit()
    #make_prediction(model, test_loader)

    for epoch in range(NUM_EPOCHS):
        print("Epoch - ", epoch)
        train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, config.DEVICE)

        # get on validation
        preds, labels,val_accuracy, val_precision, val_recall, val_f1 = check_accuracy(val_loader, model, config.DEVICE)
        print("Validation accuracy : ",val_accuracy)
        quad_score_val = cohen_kappa_score(labels, preds, weights='quadratic')
        print(f"QuadraticWeightedKappa (Validation): ", quad_score_val)
        
        # get on train
        predsontrain, labelsontrain, train_accuracy,train_precision,train_recall, train_f1 = check_accuracy(train_loader, model, config.DEVICE)
        print("Training accuracy : ",train_accuracy)
        quad_score_train = cohen_kappa_score(labelsontrain, predsontrain, weights='quadratic')
        print(f"QuadraticWeightedKappa (Train): ", quad_score_train)

        if val_accuracy > best_accuracy:
            best_train_accuracy = train_accuracy
            best_accuracy = val_accuracy
            epoch_checkpoint = epoch+1
            best_train_precision = train_precision
            best_precision = val_precision
            best_train_recall = train_recall
            best_recall = val_recall
            best_train_f1 = train_f1
            best_f1 = val_f1
            checkpoint = {
                    "state_dict": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                }
            save_checkpoint(checkpoint, filename=os.path.join(output_folder+"/b3_.pth.tar"))
        
        if early_stopper.early_stop(val_accuracy):
            print("Early stopping triggered!")
            break
        
    return epoch, best_accuracy, best_train_accuracy, best_train_precision, best_train_recall, best_train_f1, best_precision, best_recall, best_f1, quad_score_val   

In [8]:
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = os.path.join("/home/shashank/All_Code/Research/APTOS_Detection/Model_weights", f"{current_time}_DenseNet201_REALSynth_log.csv")
with open(log_file, 'w', newline='') as csvfile:
    fieldnames = ['epoch','batch_size','Learning_rate', 'training_loss', 'training_accuracy',
                  'train_precision','train_recall','train_f1','val_accuracy', 'val_loss','val_precision','val_recall','val_f1','Qudratic_kappa_score_val']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

for hyperparams in hyperparameter_combinations:
    torch.cuda.empty_cache()
    # Instantiate EarlyStopper with desired patience and min_delta
    early_stopper = EarlyStopper(patience=40, min_delta=0.001)
    print("hyperparameter_combinations >>>>>>>>>>>>>> ",hyperparams)
    epoch , best_accuracy,best_train_accuracy,best_train_precision,best_train_recall,best_train_f1, best_precision, best_recall, best_f1, quad_score_val = training_main(hyperparams,early_stopper)
    

    with open(log_file, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({
            'epoch': epoch,
            'batch_size': hyperparams['batch_size'],
            'Learning_rate': hyperparams['learning_rate'],
            'training_loss': "0",
            'training_accuracy': best_train_accuracy,
            'train_precision' : best_train_precision,
            'train_recall' : best_train_recall,
            'train_f1' : best_train_f1,
            
            'val_accuracy': best_accuracy,
            'val_loss' : "0",
            'val_precision' : best_precision,
            'val_recall' : best_recall,
            'val_f1' : best_f1,
            'Qudratic_kappa_score_val': quad_score_val,
            
                })

hyperparameter_combinations >>>>>>>>>>>>>>  {'batch_size': 64, 'learning_rate': 0.0002, 'epochs': 700}
Epoch -  0


  0%|          | 0/107 [00:00<?, ?it/s]